In [1]:
import pandas as pd 
import numpy as np

In [2]:
movie = pd.read_csv('movie.csv')
rating = pd.read_csv('rating.csv').drop("timestamp", axis = 1) # timestamp değişkenini kullanmayacağız

In [3]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating.head()

,userId,movieId,rating
0,1,2,3.5
1,1,29,3.5
2,1,32,3.5
3,1,47,3.5
4,1,50,3.5


In [5]:
movie["movieId"] = movie["movieId"].astype(object)
rating["movieId"] = rating["movieId"].astype(object)
rating["userId"] = rating["userId"].astype(object)

In [6]:
rating.isnull().sum()
movie.isnull().sum()

movieId    0
title      0
genres     0
dtype: int64

# Veriler üzerinde değişikliğe gidilmeden önce tüm oyların ortalamasını alıyoruz.

In [7]:
C = rating["rating"].mean()
C

3.5255285642993797

# Öncelikle rating verisini film Id’lerine göre kaç tane kullanıcı tarafından kaç oy alındığına dair grupluyoruz. Daha sonradan da index üzerinde düzenlemeler yapıyoruz.



In [8]:
rating_count = rating.groupby(["movieId"]).count()
rating_count["movieId"] = rating_count.index
rating_count.index.name = None
rating_count.reset_index(inplace = True)
del rating_count["index"]
rating_count

,userId,rating,movieId
0,49695,49695,1
1,22243,22243,2
2,12735,12735,3
3,2756,2756,4
4,12161,12161,5
...,...,...,...
26739,1,1,131254
26740,1,1,131256
26741,1,1,131258
26742,1,1,131260


# Şimdi de rating verisini film Id’lerine göre ortalamalarını veren yeni bir veri ortaya koyuyoruz. Daha sonradan bu veri üzerinde de index düzenlemeleri yapıyoruz.

In [9]:
means = rating.groupby(["movieId"]).mean()
means["movieId"] = means.index
means.index.name = None
means.reset_index(inplace = True)
del means["index"]
means = means.rename(columns = {"rating":"mean"}) 
means

,mean,movieId
0,3.921240,1
1,3.211977,2
2,3.151040,3
3,2.861393,4
4,3.064592,5
...,...,...
26739,4.000000,131254
26740,4.000000,131256
26741,2.500000,131258
26742,3.000000,131260


# Şimdi ise elde ettiğimiz iki veriyi merge modülü ile movieId değişkenine göre birleştiriyoruz.

In [10]:
df = pd.merge(movie, rating_count)
df = pd.merge(df, means)
df.head()

,movieId,title,genres,userId,rating,mean
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,49695,49695,3.921240
1,2,Jumanji (1995),Adventure|Children|Fantasy,22243,22243,3.211977
2,3,Grumpier Old Men (1995),Comedy|Romance,12735,12735,3.151040
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,2756,2756,2.861393
4,5,Father of the Bride Part II (1995),Comedy,12161,12161,3.064592


# 90. persentilde bir filmin aldığı oy sayısını, m olarak hesaplıyoruz.

In [11]:
m = df["userId"].quantile(0.90)
m

1412.0

# Oylama sayısına göre uygun olan filmler;

In [12]:
q_movies = df.copy().loc[df["userId"] >= m]
q_movies.shape

(2547, 6)

# Bu listede 2675 tane film yer almaktadır. Her nitelikli film için metrik hesaplanması gerekiyor. Bunun için weighted_rating() tanımlayacak ve score bu işlemi nitelikli filmlere uygulayacak.

# Her filmin ağırlıklı derecelendirmesini hesaplayan işlev;

In [13]:
def weighted_rating(x, m=m, C=C):
    v = x["userId"]
    R = x["mean"]
    return (v/(v+m) * R) + (m/(m+v) * C)

# Yeni bir özellik score tanımlayıp değerini weightted_rating() ile hesaplıyoruz. Ardından veriyi bu özelliğe göre sıralıyoruz. En iyi film sıralamasını ortaya çıkarıyoruz.



In [14]:
q_movies["score"] = q_movies.apply(weighted_rating, axis = 1)
q_movies = q_movies.sort_values("score", ascending = False)
q_movies.head(10)

,movieId,title,genres,userId,rating,mean,score
315,318,"Shawshank Redemption, The (1994)",Crime|Drama,63366,63366,4.446990,4.426905
843,858,"Godfather, The (1972)",Crime|Drama,41355,41355,4.364732,4.337025
49,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,47006,47006,4.334372,4.310784
523,527,Schindler's List (1993),Drama|War,50054,50054,4.310175,4.288648
1195,1221,"Godfather: Part II, The (1974)",Crime|Drama,27398,27398,4.275641,4.238877
895,912,Casablanca (1942),Drama|Romance,24349,24349,4.258327,4.218161
1169,1193,One Flew Over the Cuckoo's Nest (1975),Drama,29932,29932,4.248079,4.215529
887,904,Rear Window (1954),Mystery|Thriller,17449,17449,4.271334,4.215500
737,750,Dr. Strangelove or: How I Learned to Stop Worr...,Comedy|War,23220,23220,4.247287,4.205913
2873,2959,Fight Club (1999),Action|Crime|Drama|Thriller,40106,40106,4.227123,4.203262


# 2. İçeriğe Dayalı Tavsiyeler

Bu kısımda da filmlerin türlerine göre benzer filmleri veren bir tavsiye sistemi uygulayacağız.

In [15]:
df[["genres"]].head()

,genres
0,Adventure|Animation|Children|Comedy|Fantasy
1,Adventure|Children|Fantasy
2,Comedy|Romance
3,Comedy|Drama|Romance
4,Comedy


# Terim Frekansı - Ters Belge Frekansı (TF-IDF) vektörlerini hesaplayacağız.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(stop_words = "english")
df["genres"] = df["genres"].fillna("|")

tfidf_matrix = tfidf.fit_transform(df["genres"])
tfidf_matrix.shape

(25452, 23)

# Kosinüs benzerlik matrisini hesaplıyoruz.

In [17]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim

array([[1.        , 0.81206078, 0.15869798, ..., 0.        , 0.        ,
        0.        ],
       [0.81206078, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.15869798, 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

# Film başlığını giriş olarak alan ve en çok benzeyen 10 filmi veren bir işlev çıkaracağız.

In [18]:
indices = pd.Series(df.index, index = df["title"]).drop_duplicates()
indices

title
Toy Story (1995)                                     0
Jumanji (1995)                                       1
Grumpier Old Men (1995)                              2
Waiting to Exhale (1995)                             3
Father of the Bride Part II (1995)                   4
                                                 ...  
Springfield Rifle (1952)                         25447
Why Be Good? (1929)                              25448
The Expedition to the End of the World (2014)    25449
Jealousy (2013)                                  25450
The Divine Woman (1928)                          25451
Length: 25452, dtype: int64

In [19]:
def get_recommendations(title, cosine_sim = cosine_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key = lambda x : x[1], reverse = True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return df["title"].iloc[movie_indices]

In [20]:
get_recommendations("Toy Story (1995)")

2209                                           Antz (1998)
3027                                    Toy Story 2 (1999)
3663        Adventures of Rocky and Bullwinkle, The (2000)
3922                      Emperor's New Groove, The (2000)
4790                                 Monsters, Inc. (2001)
10105    DuckTales: The Movie - Treasure of the Lost La...
10977                                     Wild, The (2006)
11860                               Shrek the Third (2007)
13324                       Tale of Despereaux, The (2008)
18229    Asterix and the Vikings (Astérix et les Viking...
Name: title, dtype: object